In [13]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import tensorflow as tf 
from tensorflow import keras
#from sklearn import cross_validation, ensemble, preprocessing, metrics
from sklearn.ensemble import RandomForestClassifier   ##分類
#from sklearn.ensemble import RandomForestRegressor    ##回歸
#from sklearn.metrics import roc_auc_score
#還是要固定訓練資料和測試資料筆數
from sklearn.metrics import precision_recall_fscore_support as score

In [15]:
#all_no_miss
train = pd.read_csv('0408trans_RF_all_train.csv')

In [16]:
train['is_train'] = np.random.uniform(0, 1, len(train)) <= 0.85   #0.8
train, test = train[train['is_train']==True], train[train['is_train']==False]

In [17]:
train.reset_index( drop=True, inplace=True )
del train['is_train']
train_feature = train.columns[1:]
train_target = train.columns[0]
train_feature = train[train_feature]
train_y = train[train_target]

In [18]:
test.reset_index( drop=True, inplace=True )
del test['is_train']
test_feature = test.columns[1:]
test_target = test.columns[0]
test_feature = test[test_feature]
test_y = test[test_target]

In [19]:
train = pd.get_dummies(data = train , columns=["GENDER","occupation","WHERE","fracture_type","ANESTHESIA_TYPE",
                                               "PATIENT_ORIGIN","OP_TYPE","PRE_DIAG","POST_DIAG"])

In [20]:
column = pd.read_csv('col_all.csv')
categorical_variables = ["AGE","BMI","OP_CODE","GENDER","occupation","WHERE","fracture_type","ANESTHESIA_TYPE",
                        "PATIENT_ORIGIN","OP_TYPE","PRE_DIAG","POST_DIAG"]

In [10]:
def graph_feature_importances(model,feature_names,autoscale=True,headroom=0.05,width=10,summarized_columns=None):
    if autoscale:
        x_scale = model.feature_importances_.max() + headroom
    else:
        x_scale = 1
    
    feature_dict = dict(zip(feature_names, model.feature_importances_))
   
    if summarized_columns:
        for col_name in summarized_columns:
            sum_value = sum(x for i, x in feature_dict.items() if col_name in i )    #iteritems 改成 items  在python3
            #print(sum_value) 
            keys_to_remove = [i for i in feature_dict.keys() if col_name in i ]
            
            for i in keys_to_remove:
                feature_dict.pop(i)
                #print(i),可查看合併哪些類別
            feature_dict[col_name] = sum_value
            #print(feature_dict[col_name])  ; 可查看合併哪些類別的數值
    results = pd.Series(feature_dict, index = feature_dict.keys()) #本來就是values,因此前面feature_dict.values()改成feature_dict
    #print(results)   ;可查看原始 Series()的數值     
    new_sort = results.sort_values(ascending = True)   
    print(new_sort.sort_values(ascending = False))   #可查看排序後 Series()的數值 
    #print(sum(new_sort))   #看總和
    new_sort.plot(kind="barh", figsize= (width,len(new_sort)/2), xlim = (0,x_scale) )

In [11]:
RF_normal_100 = RandomForestClassifier(n_estimators = 100, n_jobs = -1, max_depth = None,
                                         max_features = None, oob_score = True ) 

In [12]:
RF_normal_100.fit( train_feature , train_y )

ValueError: could not convert string to float: 'A'

In [ ]:
graph_feature_importances( RF_normal_100 , column.columns, summarized_columns = categorical_variables)

In [ ]:
RF_normal_100_predict = RF_normal_100.predict(test_feature)

In [ ]:
RF_normal_100_predict

In [ ]:
pd.crosstab( test_y, RF_normal_100_predict, rownames=['Actual Species'], colnames=['Predicted Species'])

In [ ]:
RF_normal_100.score(test_feature, test_y)

In [ ]:
RF_balanced_100 = RandomForestClassifier(n_estimators = 100, n_jobs = -1, max_depth = None,
                                         max_features = None, oob_score = True, class_weight = "balanced" ) 

In [ ]:
RF_balanced_100.fit( train_feature , train_y )

In [ ]:
graph_feature_importances( RF_balanced_100 , column.columns, summarized_columns = categorical_variables)

In [ ]:
RF_balanced_100_predict = RF_balanced_100.predict(test_feature)

In [ ]:
RF_balanced_100_predict

In [ ]:
pd.crosstab( test_y, RF_balanced_100_predict, rownames=['Actual Species'], colnames=['Predicted Species'])

In [ ]:
RF_balanced_100.score(test_feature, test_y)

In [ ]:
test_array = np.array(test_y)

In [ ]:
test_array

In [ ]:
random_pain = []
for painsco in range( len(test_array) ):
    random_pain.append( np.random.choice(a = [0,1,2,3,4,5],replace = True, p=[0.05,0.09,0.49,0.32,0.04,0.01]) )
    pain_array = np.array(random_pain)

In [ ]:
random_acc = ( pain_array - test_array )
count_pain = 0 
for iter_array in range( len(random_acc) ):
    if( random_acc[iter_array] == 0 ):
        count_pain = count_pain + 1
real_acc = ( count_pain / len(random_acc) )

In [ ]:
real_acc

In [7]:
train.to_csv('train_data.csv',index = False )
test.to_csv('test_data.csv',index = False )